In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [2]:
# 1. Load the dataset
data = pd.read_csv("Alphabets_data.csv")

In [11]:
print(data.shape[0])

print(data.shape[1] - 1)

print(list(data.columns))

20000
16
['letter', 'xbox', 'ybox', 'width', 'height', 'onpix', 'xbar', 'ybar', 'x2bar', 'y2bar', 'xybar', 'x2ybar', 'xy2bar', 'xedge', 'xedgey', 'yedge', 'yedgex']


In [13]:
data.head()

letter  xbox  ybox  width  height  onpix  xbar  ybar  x2bar  y2bar  xybar  \
0      T     2     8      3       5      1     8    13      0      6      6   
1      I     5    12      3       7      2    10     5      5      4     13   
2      D     4    11      6       8      6    10     6      2      6     10   
3      N     7    11      6       6      3     5     9      4      6      4   
4      G     2     1      3       1      1     8     6      6      6      6   

   x2ybar  xy2bar  xedge  xedgey  yedge  yedgex  
0      10       8      0       8      0       8  
1       3       9      2       8      4      10  
2       3       7      3       7      3       9  
3       4      10      6      10      2       8  
4       5       9      1       7      5      10

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   letter  20000 non-null  object
 1   xbox    20000 non-null  int64 
 2   ybox    20000 non-null  int64 
 3   width   20000 non-null  int64 
 4   height  20000 non-null  int64 
 5   onpix   20000 non-null  int64 
 6   xbar    20000 non-null  int64 
 7   ybar    20000 non-null  int64 
 8   x2bar   20000 non-null  int64 
 9   y2bar   20000 non-null  int64 
 10  xybar   20000 non-null  int64 
 11  x2ybar  20000 non-null  int64 
 12  xy2bar  20000 non-null  int64 
 13  xedge   20000 non-null  int64 
 14  xedgey  20000 non-null  int64 
 15  yedge   20000 non-null  int64 
 16  yedgex  20000 non-null  int64 
dtypes: int64(16), object(1)
memory usage: 2.6+ MB


In [16]:
# Class distribution (assuming label is in the last column)

print(data.iloc[:, -1].value_counts())

yedgex
8     8047
7     3472
9     2358
6     1827
10    1578
5      992
11     868
4      478
12     137
3      130
13      49
2       30
1       17
14      13
15       2
0        2
Name: count, dtype: int64


In [18]:
data.isnull().sum()

letter    0
xbox      0
ybox      0
width     0
height    0
onpix     0
xbar      0
ybar      0
x2bar     0
y2bar     0
xybar     0
x2ybar    0
xy2bar    0
xedge     0
xedgey    0
yedge     0
yedgex    0
dtype: int64

In [21]:
from sklearn.preprocessing import StandardScaler

# Separate features and labels
X = data.drop('letter', axis=1) # Exclude 'letter' column
y = data.iloc[:, -1]   # Last column is assumed label

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [22]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

print("\nClasses:", encoder.classes_)
print("Encoded Labels:", y_encoded[:10])



Classes: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
Encoded Labels: [ 8 10  9  8 10  7 10  7  7  8]


In [23]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify=y_encoded)

print("\nTrain set size:", X_train.shape[0])
print("Test set size:", X_test.shape[0])



Train set size: 16000
Test set size: 4000


In [30]:
from tensorflow.keras.utils import to_categorical
import numpy as np

# One-hot encode the class labels
num_classes = len(np.unique(y_train))
y_train_cat = to_categorical(y_train, num_classes)
y_test_cat = to_categorical(y_test, num_classes)


In [31]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Build the model
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),  # hidden layer
    Dense(num_classes, activation='softmax')  # output layer
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print model architecture
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 64)             │         1,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 16)             │         1,040 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,128 (8.31 KB)

 Trainable params: 2,128 (8.31 KB)

 Non-trainable params: 0 (0.00 B)

In [32]:
# Train the model
history = model.fit(X_train, y_train_cat,
                    validation_data=(X_test, y_test_cat),
                    epochs=20,
                    batch_size=32,
                    verbose=1)


Epoch 1/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.4015 - loss: 1.8113 - val_accuracy: 0.6438 - val_loss: 1.0365
Epoch 2/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7036 - loss: 0.9264 - val_accuracy: 0.8200 - val_loss: 0.6970
Epoch 3/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8435 - loss: 0.6236 - val_accuracy: 0.8808 - val_loss: 0.4899
Epoch 4/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9067 - loss: 0.4409 - val_accuracy: 0.9222 - val_loss: 0.3548
Epoch 5/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9396 - loss: 0.3205 - val_accuracy: 0.9545 - val_loss: 0.2627
Epoch 6/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9524 - loss: 0.2559 - val_accuracy: 0.9672 - val_loss: 0.2035
Epoch 7/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9654 - loss: 0.1899 - val_accuracy: 0.9720 - val_loss: 0.1632
Epoch 8/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9724 - loss: 0.1533 - val_accuracy: 0.

In [33]:
# Predict probabilities for test set
y_pred_prob = model.predict(X_test)

# Convert probabilities to class labels
y_pred = np.argmax(y_pred_prob, axis=1)


125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [34]:
test_loss, test_acc = model.evaluate(X_test, y_test_cat)
print("\nTest Accuracy:", round(test_acc * 100, 2), "%")


125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9955 - loss: 0.0222

Test Accuracy: 99.42 %


In [36]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

def create_model(optimizer='adam', activation='relu', neurons=64):
    model = Sequential()
    model.add(Dense(neurons, input_dim=X_train.shape[1], activation=activation))
    model.add(Dense(neurons, activation=activation))  # optional second hidden layer
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


ModuleNotFoundError: No module named 'tensorflow.keras.wrappers.scikit_learn'

In [37]:
!pip install scikeras

In [39]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, SGD
from scikeras.wrappers import KerasClassifier

def create_model(optimizer='adam', activation='relu', neurons=64):
    model = Sequential()
    model.add(Dense(neurons, input_dim=X_train.shape[1], activation=activation))
    model.add(Dense(neurons, activation=activation))  # optional second hidden layer
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model



In [40]:
from sklearn.model_selection import GridSearchCV

# Wrap with KerasClassifier
model = KerasClassifier(build_fn=create_model, verbose=0)


In [41]:
param_grid = {
    'batch_size': [16, 32],
    'epochs': [20, 30],
    'optimizer': ['adam', 'sgd'],
    'activation': ['relu', 'tanh'],
    'neurons': [32, 64]
}


In [43]:
grid = GridSearchCV(estimator=model,
                    param_grid=param_grid,
                    n_jobs=-1,
                    cv=3,
                    scoring='accuracy')

# Fit on training data
grid_result = grid.fit(X_train, y_train)

AttributeError: 'super' object has no attribute '__sklearn_tags__'

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt

# Define model builder for Keras Tuner
def model_builder_kt(hp):
    model = keras.Sequential()

    # Tune number of units in first Dense layer
    hp_units = hp.Choice('units', values=[32, 64]) # Reduced search space based on param_grid
    model.add(layers.Dense(units=hp_units, activation=hp.Choice('activation', ['relu', 'tanh']), input_shape=(X_train.shape[1],)))

    # Optional second hidden layer based on the structure in create_model
    # You can add tuning for the number of neurons in this layer as well if desired
    # hp_units_2 = hp.Choice('units_2', values=[32, 64])
    # model.add(layers.Dense(units=hp_units_2, activation=hp.Choice('activation_2', ['relu', 'tanh'])))


    model.add(layers.Dense(num_classes, activation='softmax'))

    # Tune the optimizer
    hp_optimizer = hp.Choice('optimizer', values=['adam', 'sgd'])

    model.compile(
        optimizer=hp_optimizer, # Use the tuned optimizer
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# Initialize the Keras Tuner
tuner = kt.RandomSearch(
    model_builder_kt,
    objective='val_accuracy',
    max_trials=5,  # You can adjust the number of trials
    executions_per_trial=1, # Number of models to build and train for each trial
    directory='keras_tuner_dir', # Directory to save results
    project_name='alphabet_classification_keras_tuner' # Project name
)

# Run the hyperparameter search
# Use the one-hot encoded labels (y_train_cat) for Keras Tuner
tuner.search(X_train, y_train_cat, epochs=20, validation_split=0.2, verbose=1)

# Get the best hyperparameters and best model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print("\nBest hyperparameters found by Keras Tuner:")
print(f"Units in first layer: {best_hps.get('units')}")
print(f"Activation: {best_hps.get('activation')}")
print(f"Optimizer: {best_hps.get('optimizer')}")
# If you added a second hidden layer and tuned its units/activation, print those as well
# print(f"Units in second layer: {best_hps.get('units_2')}")
# print(f"Activation in second layer: {best_hps.get('activation_2')}")

# Build the best model with the found hyperparameters
best_model_kt = tuner.hypermodel.build(best_hps)

# Train the best model
print("\nTraining the best model found by Keras Tuner...")
history_kt = best_model_kt.fit(X_train, y_train_cat, epochs=20, validation_split=0.2, verbose=1)

# Evaluate the best model on the test set
print("\nEvaluating the best model on the test set...")
test_loss_kt, test_acc_kt = best_model_kt.evaluate(X_test, y_test_cat, verbose=0)
print(f"Test Accuracy of Tuned Model (Keras Tuner): {test_acc_kt:.4f}")

Trial 2 Complete [00h 00m 37s]
val_accuracy: 0.9325000047683716

Best val_accuracy So Far: 0.979687511920929
Total elapsed time: 00h 01m 24s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
64                |32                |units
relu              |tanh              |activation
adam              |adam              |optimizer

Epoch 1/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.3566 - loss: 2.0098 - val_accuracy: 0.6175 - val_loss: 1.1127
Epoch 2/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.6561 - loss: 1.0251 - val_accuracy: 0.7741 - val_loss: 0.8113
Epoch 3/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7984 - loss: 0.7414 - val_accuracy: 0.8459 - val_loss: 0.6104
Epoch 4/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8807 - loss: 0.5385 - val_accuracy: 0.9091 - val_loss: 0.4573
Epoch 5/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9171 - loss: 0.4110 - val_accuracy: 0.9344 - val_loss: 0.35

KeyboardInterrupt: 